In [1]:
import pandas as pd
import re
import numpy as np

#TF/TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Pierre-
[nltk_data]     François\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Pierre-
[nltk_data]     François\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Pierre-
[nltk_data]     François\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Load product page detail data
#product_page_df = pd.read_csv("D:/KU Leuven/Master Thesis/Starting-over-2/Analysis/cleaning/clean/PRODUCT.csv")
product_page_df = pd.read_csv("C:/Users/Pierre-François/ChiChi's Research/ChiChi/product_page.csv")
# Load search result page data
search_result_df = pd.read_csv("C:/Users/Pierre-François/ChiChi's Research/ChiChi/search_engine_result_page.csv")

In [10]:
product_page_df.info()
search_result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32399 entries, 0 to 32398
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   asin           32399 non-null  object
 1   search_term    32399 non-null  object
 2   scrape_date    32399 non-null  int64 
 3   title          32399 non-null  object
 4   brand          32217 non-null  object
 5   categories     32399 non-null  object
 6   bullet_points  27355 non-null  object
 7   description    27868 non-null  object
 8   bsr_1          25585 non-null  object
 9   bsr_2          25548 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1703716 entries, 0 to 1703715
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   asin            object 
 1   search_term     object 
 2   page_number     int64  
 3   position        int64  
 4   scrape_date     object 
 5   scra

In [8]:
import pandas as pd
import os
from datetime import datetime

# Load the CSV file
def split_csv_by_date(input_file, date_column, output_folder='split_files'):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Convert date column to datetime if it's not already
    if not pd.api.types.is_datetime64_any_dtype(df[date_column]):
        df[date_column] = pd.to_datetime(df[date_column])
    
    # Group by date (year-month-day)
    grouped = df.groupby(df[date_column].dt.strftime('%Y-%m-%d'))
    
    # Save each group to a separate file
    for date, group in grouped:
        output_file = os.path.join(output_folder, f'data_{date}.csv')
        group.to_csv(output_file, index=False)
        print(f'Created file: {output_file} with {len(group)} rows')

# Example usage
split_csv_by_date("C:/Users/Pierre-François/ChiChi's Research/ChiChi/search_engine_result_page.csv", 'scrape_date')

Created file: split_files\data_2025-03-20.csv with 180 rows
Created file: split_files\data_2025-03-21.csv with 231788 rows
Created file: split_files\data_2025-03-22.csv with 251861 rows
Created file: split_files\data_2025-03-23.csv with 254258 rows
Created file: split_files\data_2025-03-24.csv with 248231 rows
Created file: split_files\data_2025-03-25.csv with 246877 rows
Created file: split_files\data_2025-03-26.csv with 228048 rows
Created file: split_files\data_2025-03-27.csv with 242473 rows


In [4]:
product_page = product_page_df
search_result = search_result_df
print(product_page.shape)
print(search_result_df.shape)

(32399, 10)
(1703716, 15)


In [5]:
#convert scrape date to date time format
search_result['scrape_date'] = pd.to_datetime(search_result['scrape_date'])
print(search_result['scrape_date'].info())

# #remove rows where scrape_hour is 15 on scrape_date 
search_result['scrape_date'] = pd.to_datetime(search_result['scrape_date'])
search_result = search_result[~((search_result['scrape_date']=='23/03/2025')&(search_result['scrape_hour']==15))]
search_result = search_result[search_result['scrape_date'].dt.date != pd.to_datetime('2025-03-20').date()]

<class 'pandas.core.series.Series'>
RangeIndex: 1703716 entries, 0 to 1703715
Series name: scrape_date
Non-Null Count    Dtype         
--------------    -----         
1703716 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 13.0 MB
None


In [6]:
remove_term = [
    'roku', 'coffee','gift cards','pens','water','apple watch band','amazon gift card',
    'queen bed frame','xbox gift card','kindle unlimited free books','tide pods','glue sticks'
]
print(len(remove_term))

12


In [7]:
#filter out rows without the search term
for term in remove_term:
    product_page = product_page[~product_page['search_term'].str.contains(term, case=False,na=False)]
for term in remove_term:
    search_result = search_result[~search_result['search_term'].str.contains(term, case=False,na=False)]

In [8]:
search_result = search_result.drop_duplicates(subset=['asin', 'search_term', 'page_number', 'position', 'scrape_date', 'scrape_hour'], keep='first')
print(search_result.shape)
product_page = product_page.drop_duplicates(subset=['asin', 'search_term'], keep='first')
print(product_page.shape)

(1523052, 15)
(29341, 10)


In [9]:
merged_data = pd.merge(
    search_result,
    product_page[['asin','brand','bsr_1','bsr_2','categories','bullet_points','description']],
    on='asin',     
    how='left', ) 
print(merged_data.head(5))

         asin   search_term  page_number  position scrape_date scrape_time  \
0  B0D3HD4KTD  aa batteries            1         1  2025-03-23    16-06:03   
1  B0CJ399BX5  aa batteries            1         2  2025-03-23    16-06:03   
2  B00E3DVQFS  aa batteries            1         3  2025-03-23    16-06:03   
3  B0CCR5QJYS  aa batteries            1         4  2025-03-23    16-06:03   
4  B093LVB4P7  aa batteries            1         5  2025-03-23    16-06:03   

   scrape_hour                                              title  price  \
0           16  Logitech G 309 LIGHTSPEED, Lightweight Wireles...  64.59   
1           16  Midland PB42 2800MAH Lithium Battery for Alan ...  64.95   
2           16  Duracell Rechargeable AA batteries (pack of 4 ...  11.19   
3           16  Delyeepow 4 Pack USB-C AA AAA Lithium 1.5V 340...  36.99   
4           16  Duracell Plus AA Batteries (Pack of 12) - Alka...   9.99   

   original_price  ...  reviews_count  rating              sales_history  

In [10]:
merged_data = merged_data[~merged_data['categories'].isnull()]

In [11]:
print(merged_data.shape)
print("Shape before merge:", search_result.shape)
print("Shape after merge:", merged_data.shape)
print(merged_data.head(5))

(1546956, 21)
Shape before merge: (1523052, 15)
Shape after merge: (1546956, 21)
         asin   search_term  page_number  position scrape_date scrape_time  \
0  B0D3HD4KTD  aa batteries            1         1  2025-03-23    16-06:03   
1  B0CJ399BX5  aa batteries            1         2  2025-03-23    16-06:03   
2  B00E3DVQFS  aa batteries            1         3  2025-03-23    16-06:03   
3  B0CCR5QJYS  aa batteries            1         4  2025-03-23    16-06:03   
4  B093LVB4P7  aa batteries            1         5  2025-03-23    16-06:03   

   scrape_hour                                              title  price  \
0           16  Logitech G 309 LIGHTSPEED, Lightweight Wireles...  64.59   
1           16  Midland PB42 2800MAH Lithium Battery for Alan ...  64.95   
2           16  Duracell Rechargeable AA batteries (pack of 4 ...  11.19   
3           16  Delyeepow 4 Pack USB-C AA AAA Lithium 1.5V 340...  36.99   
4           16  Duracell Plus AA Batteries (Pack of 12) - Alka...   9.

In [12]:
#get the product cateory
merged_data['main_cat'] = merged_data['categories'].apply(lambda x: x.split("|")[0] if pd.notnull(x) else x)
merged_data['sub_cat'] = merged_data['categories'].apply(
    lambda x: x.split("|")[2] if pd.notnull(x) and len(x.split("|")) > 2 else x)
print(merged_data[['search_term','main_cat','sub_cat']])

          search_term           main_cat                 sub_cat
0        aa batteries        Electronics             Accessories
1        aa batteries        Electronics  Rechargeable Batteries
2        aa batteries        Electronics  Rechargeable Batteries
3        aa batteries        Electronics  Rechargeable Batteries
4        aa batteries        Electronics    Disposable Batteries
...               ...                ...                     ...
1562871      yoga mat  Sports & Outdoors                    Yoga
1562872      yoga mat  Sports & Outdoors                    Yoga
1562873      yoga mat  Sports & Outdoors                    Yoga
1562874      yoga mat  Sports & Outdoors                    Yoga
1562875      yoga mat  Sports & Outdoors             Accessories

[1546956 rows x 3 columns]


In [13]:
#Transform bsr to extract main_cat_rank and sub_cat_rank
merged_data['bsr_1'] = merged_data['bsr_1'].str.replace(',', '')
merged_data['bsr_2'] = merged_data['bsr_2'].str.replace(',', '')
merged_data['main_cat_rank'] = merged_data['bsr_1'].str.extract(r'(\d+)')
merged_data['main_cat_rank'] = merged_data['main_cat_rank'].fillna(9999999).astype(int)
merged_data['sub_cat_rank'] = merged_data['bsr_2'].str.extract(r'(\d+)')
merged_data['sub_cat_rank'] = merged_data['sub_cat_rank'].fillna(9999999).astype(int)
print(merged_data[['asin','main_cat_rank']])
print(merged_data[['asin','sub_cat_rank']])


               asin  main_cat_rank
0        B0D3HD4KTD          75121
1        B0CJ399BX5         346664
2        B00E3DVQFS            180
3        B0CCR5QJYS          13057
4        B093LVB4P7             14
...             ...            ...
1562871  B0B5195NZV        9999999
1562872  B08JV4HF3N        9999999
1562873  B0CD9W8WMX         164881
1562874  B0DLL7NF5S        9999999
1562875  B0CJ8SFMT8           1596

[1546956 rows x 2 columns]
               asin  sub_cat_rank
0        B0D3HD4KTD           662
1        B0CJ399BX5          1853
2        B00E3DVQFS             3
3        B0CCR5QJYS           116
4        B093LVB4P7             2
...             ...           ...
1562871  B0B5195NZV       9999999
1562872  B08JV4HF3N       9999999
1562873  B0CD9W8WMX           925
1562874  B0DLL7NF5S       9999999
1562875  B0CJ8SFMT8             3

[1546956 rows x 2 columns]


In [14]:
#transform past_month_sale to categorical bucket
def bucket_sales(text):
    if pd.isna(text):
        return "<50"
    num = re.findall(r'(\d+)', text)
    if not num:
        return np.nan
    num = int(num[0])
    if num < 50:
        return "<50"
    elif num < 100:
        return "50-100"
    elif num < 200:
        return "100-200"
    elif num < 300:
        return "200-300"
    elif num < 500:
        return "300-500"
    elif num < 1000:
        return "500-1000"
    else:
        return ">1000"
merged_data['sales_bucket'] = merged_data['sales_history'].apply(bucket_sales)

In [15]:
merged_data['sales_badge'] = np.where(merged_data['sales_history'].isnull(), 0, 1)
merged_data['discount'] = np.where(merged_data['original_price'].isnull(), 0, 1)
print(merged_data['sales_badge'])
print(merged_data['discount'])

0          0
1          0
2          1
3          0
4          1
          ..
1562871    0
1562872    0
1562873    0
1562874    0
1562875    0
Name: sales_badge, Length: 1546956, dtype: int64
0          1
1          0
2          1
3          0
4          1
          ..
1562871    0
1562872    0
1562873    0
1562874    0
1562875    0
Name: discount, Length: 1546956, dtype: int64


In [16]:
merged_data['reviews_count']=merged_data['reviews_count'].fillna(0)
merged_data['rating']=merged_data['rating'].fillna(0)
#clean the row where there is review but no rating
merged_data.loc[merged_data['rating'].isnull(), 'reviews_count'] = np.nan
merged_data['reviews_count']=merged_data['reviews_count'].astype(int)
print(merged_data[['reviews_count','rating']])

         reviews_count  rating
0                   67     4.5
1                   18     4.0
2                93670     4.5
3                  151     4.2
4               208267     4.8
...                ...     ...
1562871            179     4.3
1562872           3549     4.4
1562873             63     4.5
1562874              2     5.0
1562875             85     4.7

[1546956 rows x 2 columns]


In [17]:
merged_data = merged_data[~merged_data['price'].isnull()]

In [18]:
#log transformation
merged_data['log_sub_rank'] = np.log1p(merged_data['sub_cat_rank'])
merged_data['log_main_rank'] = np.log1p(merged_data['main_cat_rank'])
merged_data['log_price'] = np.log1p(merged_data['price'].fillna(0))
merged_data['log_review'] = np.log1p(merged_data['reviews_count'])

In [21]:
merged_data['popularity'] = (
    (merged_data['sub_cat_rank']<= merged_data['sub_cat_rank'].quantile(0.20)) &
    (merged_data['main_cat_rank'] <= merged_data['main_cat_rank'].quantile(0.20)) &
    (merged_data['rating'] <= merged_data['main_cat_rank'].quantile(0.20)) &
    (merged_data['sales_history'].fillna(0) != 0) &
    (merged_data['reviews_count'] >= merged_data['reviews_count'].quantile(0.80))
)

In [22]:
# Assuming you have a 'query_group' column that identifies each search query group
merged_data['popularity_1'] = merged_data.groupby('search_term').apply(
    lambda group: (
        # Lower rank = better (top 20% within this query group)
        (group['sub_cat_rank'] <= group['sub_cat_rank'].quantile(0.20)) &
        (group['main_cat_rank'] <= group['main_cat_rank'].quantile(0.20)) &
        # Higher rating = better (top 20% within this query group)
        (group['rating'] >= group['rating'].quantile(0.80)) &
        # Has sales history
        (group['sales_history'].fillna(0) != 0) &
        # Higher review count = better (top 20% within this query group)
        (group['reviews_count'] >= group['reviews_count'].quantile(0.80))
    )
).reset_index(level=0, drop=True)

C:\Users\Pierre-François\AppData\Local\Temp\ipykernel_21404\529018329.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  (group['sales_history'].fillna(0) != 0) &
C:\Users\Pierre-François\AppData\Local\Temp\ipykernel_21404\529018329.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  (group['sales_history'].fillna(0) != 0) &
C:\Users\Pierre-François\AppData\Local\Temp\ipykernel_21404\529018329.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(co

In [23]:
print(merged_data['popularity_1'])

0          False
1          False
2          False
3          False
4           True
           ...  
1562871    False
1562872    False
1562873    False
1562874    False
1562875    False
Name: popularity_1, Length: 1516978, dtype: bool


In [24]:
#calculate discount percentage and bucket it
def discount_group(row):
    if pd.isna(row['original_price']) or row['original_price'] == 0:
        return "no_discount"
    discount = (row['original_price'] - row['price']) / row['original_price'] * 100
    if discount == 0:
        return "no_discount"
    elif discount <= 10:
        return "0-10%"
    elif discount <= 30:
        return "10-30%"
    elif discount <= 40:
        return "30-40%"
    elif discount <= 50:
        return "40-50%"
    else:
        return ">50%"
merged_data['discount_group'] = merged_data.apply(discount_group, axis=1)

In [25]:
merged_data['group_id'] = merged_data['search_term'].astype(str) + '_' + merged_data['scrape_date'].astype(str) + '_' + merged_data['scrape_hour'].astype(str)
merged_data['group_id'] = merged_data['group_id'].astype('category').cat.codes
merged_data['price_competitiveness'] = merged_data.groupby('group_id')['price'].transform(
    lambda x: 1 - (x / x.max()).clip(0, 1))

In [26]:
merged_data['continuous_rank'] = (merged_data['page_number'] - 1) * 60 + merged_data['position']

In [27]:
merged_data = merged_data.drop_duplicates(subset=['asin', 'search_term', 'page_number', 'position', 'scrape_date', 'scrape_hour'], keep='first')
print(merged_data.shape)

(1477644, 38)


In [28]:
print(merged_data.isnull().sum())
print(merged_data.info())

asin                           0
search_term                    0
page_number                    0
position                       0
scrape_date                    0
scrape_time                    0
scrape_hour                    0
title                          0
price                          0
original_price           1131790
sponsored                      0
reviews_count                  0
rating                         0
sales_history            1381661
prime                          0
brand                       8280
bsr_1                     202702
bsr_2                     203510
categories                     0
bullet_points             132751
description               177596
main_cat                       0
sub_cat                        0
main_cat_rank                  0
sub_cat_rank                   0
sales_bucket                   0
sales_badge                    0
discount                       0
log_sub_rank                   0
log_main_rank                  0
log_price 

In [194]:
# Group by scrape_date and save each group as a separate CSV
#for date, group in merged_data.groupby('scrape_date'):
    #date_str = pd.to_datetime(date).strftime('%Y-%m-%d')  # Format the date nicely
    #group.to_csv(f'D:/KU Leuven/Master Thesis/Starting-over-2/Analysis/cleaning/clean_{date_str}.csv', index=False)

In [29]:
daily_counts = merged_data.groupby(merged_data['scrape_date'].dt.date).size()
print(daily_counts)

scrape_date
2025-03-21    200895
2025-03-22    218328
2025-03-23    220936
2025-03-24    215740
2025-03-25    215211
2025-03-26    197561
2025-03-27    208973
dtype: int64


In [30]:
all_data = merged_data
sponsored = merged_data[merged_data['sponsored']==False]
print(all_data.shape)
print(sponsored.shape)
#sponsored_filtered = merged_data[merged_data['sponsored']==True]

(1477644, 38)
(1194473, 38)


In [31]:
all_data.to_csv("C:/Users/Pierre-François/ChiChi's Research/ChiChi/test_all.csv")
sponsored.to_csv("C:/Users/Pierre-François/ChiChi's Research/ChiChi/full_organic.csv")

In [152]:
print(all_data.shape)

(1516978, 26)
